In [1]:
from tensorflow.keras.models import load_model
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import numpy as np
from sklearn.impute import SimpleImputer

In [17]:
# Load the model
loaded_model = load_model('lstm_model.h5')

data = pd.read_csv('Sample_cic1.csv')
data.head()

,src_ip,dst_ip,src_port,dst_port,protocol,timestamp,flow_duration,flow_byts_s,flow_pkts_s,fwd_pkts_s,...,bwd_pkts_b_avg,fwd_blk_rate_avg,bwd_blk_rate_avg,fwd_seg_size_avg,bwd_seg_size_avg,cwe_flag_count,subflow_fwd_pkts,subflow_bwd_pkts,subflow_fwd_byts,subflow_bwd_byts
0,172.16.0.5,192.168.50.1,60675,80,6,12/1/2018 17:17,5221153,1.525717e+03,4.979743,2.681400,...,0.0,0.0,0.0,204.857143,424.833333,0,14,12,2868,5098
1,172.16.0.5,192.168.50.1,60676,80,6,12/1/2018 17:17,12644553,6.516640e+01,0.949025,0.632683,...,0.0,0.0,0.0,68.000000,70.000000,0,8,4,544,280
2,65.55.163.78,192.168.50.7,443,50458,6,12/1/2018 17:17,3,4.000000e+07,666666.666700,666666.666700,...,0.0,0.0,0.0,60.000000,0.000000,0,2,0,120,0
3,65.55.163.78,192.168.50.7,443,50465,6,12/1/2018 17:17,3,4.000000e+07,666666.666700,666666.666700,...,0.0,0.0,0.0,60.000000,0.000000,0,2,0,120,0
4,192.168.50.6,172.217.3.110,56085,80,6,12/1/2018 17:17,34880627,3.921948e+01,0.630723,0.344031,...,0.0,0.0,0.0,60.000000,64.800000,0,12,10,720,648


In [19]:
data.columns = [col.strip() for col in data.columns]
rename_mapping = {
    'src_ip' : 'Source IP',
    'dst_ip' : 'Destination IP',
    'src_port': 'Source Port',
    'dst_port': 'Destination Port',
    'protocol': 'Protocol',
    'flow_duration': 'Flow Duration',
    'tot_fwd_pkts': 'Total Fwd Packets',
    'tot_bwd_pkts': 'Total Backward Packets',
    'totlen_fwd_pkts': 'Total Length of Fwd Packets',
    'totlen_bwd_pkts': 'Total Length of Bwd Packets',
    'fwd_pkt_len_max': 'Fwd Packet Length Max',
    'fwd_pkt_len_min': 'Fwd Packet Length Min',
    'fwd_pkt_len_mean': 'Fwd Packet Length Mean',
    'fwd_pkt_len_std': 'Fwd Packet Length Std',
    'bwd_pkt_len_max': 'Bwd Packet Length Max',
    'bwd_pkt_len_min': 'Bwd Packet Length Min',
    'bwd_pkt_len_mean': 'Bwd Packet Length Mean',
    'bwd_pkt_len_std': 'Bwd Packet Length Std',
    'flow_byts_s': 'Flow Bytes/s',
    'flow_pkts_s': 'Flow Packets/s',
    'flow_iat_mean': 'Flow IAT Mean',
    'flow_iat_std': 'Flow IAT Std',
    'flow_iat_max': 'Flow IAT Max',
    'flow_iat_min': 'Flow IAT Min',
    'fwd_iat_tot': 'Fwd IAT Total',
    'fwd_iat_mean': 'Fwd IAT Mean',
    'fwd_iat_std': 'Fwd IAT Std',
    'fwd_iat_max': 'Fwd IAT Max',
    'fwd_iat_min': 'Fwd IAT Min',
    'bwd_iat_tot': 'Bwd IAT Total',
    'bwd_iat_mean': 'Bwd IAT Mean',
    'bwd_iat_std': 'Bwd IAT Std',
    'bwd_iat_max': 'Bwd IAT Max',
    'bwd_iat_min': 'Bwd IAT Min',
    'fwd_psh_flags': 'Fwd PSH Flags',
    'bwd_psh_flags': 'Bwd PSH Flags',
    'fwd_urg_flags': 'Fwd URG Flags',
    'bwd_urg_flags': 'Bwd URG Flags',
    'fwd_header_len': 'Fwd Header Length',
    'bwd_header_len': 'Bwd Header Length',
    'fwd_pkts_s': 'Fwd Packets/s',
    'bwd_pkts_s': 'Bwd Packets/s',
    'pkt_len_min': 'Min Packet Length',
    'pkt_len_max': 'Max Packet Length',
    'pkt_len_mean': 'Packet Length Mean',
    'pkt_len_std': 'Packet Length Std',
    'pkt_len_var': 'Packet Length Variance',
    'fin_flag_cnt': 'FIN Flag Count',
    'syn_flag_cnt': 'SYN Flag Count',
    'rst_flag_cnt': 'RST Flag Count',
    'psh_flag_cnt': 'PSH Flag Count',
    'ack_flag_cnt': 'ACK Flag Count',
    'urg_flag_cnt': 'URG Flag Count',
    'ece_flag_cnt': 'ECE Flag Count',
    'down_up_ratio': 'Down/Up Ratio',
    'pkt_size_avg': 'Average Packet Size',
    'fwd_seg_size_avg': 'Avg Fwd Segment Size',
    'bwd_seg_size_avg': 'Avg Bwd Segment Size',
    'fwd_byts_b_avg': 'Fwd Avg Bytes/Bulk',
    'fwd_pkts_b_avg': 'Fwd Avg Packets/Bulk',
    'fwd_blk_rate_avg': 'Fwd Avg Bulk Rate',
    'bwd_byts_b_avg': 'Bwd Avg Bytes/Bulk',
    'bwd_pkts_b_avg': 'Bwd Avg Packets/Bulk',
    'bwd_blk_rate_avg': 'Bwd Avg Bulk Rate',
    'subflow_fwd_pkts': 'Subflow Fwd Packets',
    'subflow_fwd_byts': 'Subflow Fwd Bytes',
    'subflow_bwd_pkts': 'Subflow Bwd Packets',
    'subflow_bwd_byts': 'Subflow Bwd Bytes',
    'init_fwd_win_byts': 'Init_Win_bytes_forward',
    'init_bwd_win_byts': 'Init_Win_bytes_backward',
    'fwd_act_data_pkts': 'act_data_pkt_fwd',
    'fwd_seg_size_min': 'min_seg_size_forward',
    'active_mean': 'Active Mean',
    'active_std': 'Active Std',
    'active_max': 'Active Max',
    'active_min': 'Active Min',
    'idle_mean': 'Idle Mean',
    'idle_std': 'Idle Std',
    'idle_max': 'Idle Max',
    'idle_min': 'Idle Min',
    'cwe_flag_count': 'CWE Flag Count'
}

data.rename(columns=rename_mapping, inplace=True)

print(data.columns)

Index(['Source IP', 'Destination IP', 'Source Port', 'Destination Port',
       'Protocol', 'timestamp', 'Flow Duration', 'Flow Bytes/s',
       'Flow Packets/s', 'Fwd Packets/s', 'Bwd Packets/s', 'Total Fwd Packets',
       'Total Backward Packets', 'Total Length of Fwd Packets',
       'Total Length of Bwd Packets', 'Fwd Packet Length Max',
       'Fwd Packet Length Min', 'Fwd Packet Length Mean',
       'Fwd Packet Length Std', 'Bwd Packet Length Max',
       'Bwd Packet Length Min', 'Bwd Packet Length Mean',
       'Bwd Packet Length Std', 'Max Packet Length', 'Min Packet Length',
       'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance',
       'Fwd Header Length', 'Bwd Header Length', 'min_seg_size_forward',
       'act_data_pkt_fwd', 'Flow IAT Mean', 'Flow IAT Max', 'Flow IAT Min',
       'Flow IAT Std', 'Fwd IAT Total', 'Fwd IAT Max', 'Fwd IAT Min',
       'Fwd IAT Mean', 'Fwd IAT Std', 'Bwd IAT Total', 'Bwd IAT Max',
       'Bwd IAT Min', 'Bwd IAT Mean', 'Bwd 

In [4]:
data = data.drop(['Source IP','Destination IP','timestamp'], axis=1)

In [5]:
data = data.apply(pd.to_numeric, errors='coerce')

In [6]:
imputer = SimpleImputer(strategy='mean')
data = pd.DataFrame(imputer.fit_transform(data), columns=data.columns)

In [7]:
scaler = MinMaxScaler()
data = scaler.fit_transform(data)

data = np.reshape(data, (data.shape[0], 1, data.shape[1]))

In [8]:
predictions = loaded_model.predict(data)

predictions = ["DDoS" if pred > 0.5 else "Normal" for pred in predictions]

data = pd.read_csv('Sample_cic1.csv')

# Add predictions as a new column to the DataFrame
data['predictions'] = predictions

data

2/2 [==============================] - 2s 7ms/step


,src_ip,dst_ip,src_port,dst_port,protocol,timestamp,flow_duration,flow_byts_s,flow_pkts_s,fwd_pkts_s,...,fwd_blk_rate_avg,bwd_blk_rate_avg,fwd_seg_size_avg,bwd_seg_size_avg,cwe_flag_count,subflow_fwd_pkts,subflow_bwd_pkts,subflow_fwd_byts,subflow_bwd_byts,predictions
0,172.16.0.5,192.168.50.1,60675,80,6,12/1/2018 17:17,5221153,1.525717e+03,4.979743,2.681400,...,0.000000e+00,0.000000e+00,204.857143,424.833333,0,14,12,2868,5098,Normal
1,172.16.0.5,192.168.50.1,60676,80,6,12/1/2018 17:17,12644553,6.516640e+01,0.949025,0.632683,...,0.000000e+00,0.000000e+00,68.000000,70.000000,0,8,4,544,280,Normal
2,65.55.163.78,192.168.50.7,443,50458,6,12/1/2018 17:17,3,4.000000e+07,666666.666700,666666.666700,...,0.000000e+00,0.000000e+00,60.000000,0.000000,0,2,0,120,0,DDoS
3,65.55.163.78,192.168.50.7,443,50465,6,12/1/2018 17:17,3,4.000000e+07,666666.666700,666666.666700,...,0.000000e+00,0.000000e+00,60.000000,0.000000,0,2,0,120,0,DDoS
4,192.168.50.6,172.217.3.110,56085,80,6,12/1/2018 17:17,34880627,3.921948e+01,0.630723,0.344031,...,0.000000e+00,0.000000e+00,60.000000,64.800000,0,12,10,720,648,Normal
5,192.168.50.6,72.21.91.29,56123,80,6,12/1/2018 17:17,35641329,3.838241e+01,0.617261,0.336688,...,0.000000e+00,0.000000e+00,60.000000,64.800000,0,12,10,720,648,Normal
6,192.168.50.6,8.43.72.98,56133,443,6,12/1/2018 17:17,35583848,4.783069e+01,0.730669,0.393437,...,2.950000e+07,1.224490e+05,66.714286,64.000000,0,14,12,934,768,Normal
7,172.16.0.5,192.168.50.1,53948,80,6,12/1/2018 17:17,5009207,8.488370e+02,3.992648,1.996324,...,0.000000e+00,0.000000e+00,143.000000,282.200000,0,10,10,1430,2822,Normal
8,65.55.163.78,192.168.50.7,443,50466,6,12/1/2018 17:17,2,6.000000e+07,1000000.000000,1000000.000000,...,0.000000e+00,0.000000e+00,60.000000,0.000000,0,2,0,120,0,DDoS
9,65.55.163.78,192.168.50.7,443,50457,6,12/1/2018 17:17,2,6.000000e+07,1000000.000000,1000000.000000,...,0.000000e+00,0.000000e+00,60.000000,0.000000,0,2,0,120,0,DDoS


In [9]:
data.columns = [col.strip() for col in data.columns]
rename_mapping = {
    'src_ip' : 'Source IP',
    'dst_ip' : 'Destination IP',
    'src_port': 'Source Port',
    'dst_port': 'Destination Port',
    'protocol': 'Protocol',
    'flow_duration': 'Flow Duration',
    'tot_fwd_pkts': 'Total Fwd Packets',
    'tot_bwd_pkts': 'Total Backward Packets',
    'totlen_fwd_pkts': 'Total Length of Fwd Packets',
    'totlen_bwd_pkts': 'Total Length of Bwd Packets',
    'fwd_pkt_len_max': 'Fwd Packet Length Max',
    'fwd_pkt_len_min': 'Fwd Packet Length Min',
    'fwd_pkt_len_mean': 'Fwd Packet Length Mean',
    'fwd_pkt_len_std': 'Fwd Packet Length Std',
    'bwd_pkt_len_max': 'Bwd Packet Length Max',
    'bwd_pkt_len_min': 'Bwd Packet Length Min',
    'bwd_pkt_len_mean': 'Bwd Packet Length Mean',
    'bwd_pkt_len_std': 'Bwd Packet Length Std',
    'flow_byts_s': 'Flow Bytes/s',
    'flow_pkts_s': 'Flow Packets/s',
    'flow_iat_mean': 'Flow IAT Mean',
    'flow_iat_std': 'Flow IAT Std',
    'flow_iat_max': 'Flow IAT Max',
    'flow_iat_min': 'Flow IAT Min',
    'fwd_iat_tot': 'Fwd IAT Total',
    'fwd_iat_mean': 'Fwd IAT Mean',
    'fwd_iat_std': 'Fwd IAT Std',
    'fwd_iat_max': 'Fwd IAT Max',
    'fwd_iat_min': 'Fwd IAT Min',
    'bwd_iat_tot': 'Bwd IAT Total',
    'bwd_iat_mean': 'Bwd IAT Mean',
    'bwd_iat_std': 'Bwd IAT Std',
    'bwd_iat_max': 'Bwd IAT Max',
    'bwd_iat_min': 'Bwd IAT Min',
    'fwd_psh_flags': 'Fwd PSH Flags',
    'bwd_psh_flags': 'Bwd PSH Flags',
    'fwd_urg_flags': 'Fwd URG Flags',
    'bwd_urg_flags': 'Bwd URG Flags',
    'fwd_header_len': 'Fwd Header Length',
    'bwd_header_len': 'Bwd Header Length',
    'fwd_pkts_s': 'Fwd Packets/s',
    'bwd_pkts_s': 'Bwd Packets/s',
    'pkt_len_min': 'Min Packet Length',
    'pkt_len_max': 'Max Packet Length',
    'pkt_len_mean': 'Packet Length Mean',
    'pkt_len_std': 'Packet Length Std',
    'pkt_len_var': 'Packet Length Variance',
    'fin_flag_cnt': 'FIN Flag Count',
    'syn_flag_cnt': 'SYN Flag Count',
    'rst_flag_cnt': 'RST Flag Count',
    'psh_flag_cnt': 'PSH Flag Count',
    'ack_flag_cnt': 'ACK Flag Count',
    'urg_flag_cnt': 'URG Flag Count',
    'ece_flag_cnt': 'ECE Flag Count',
    'down_up_ratio': 'Down/Up Ratio',
    'pkt_size_avg': 'Average Packet Size',
    'fwd_seg_size_avg': 'Avg Fwd Segment Size',
    'bwd_seg_size_avg': 'Avg Bwd Segment Size',
    'fwd_byts_b_avg': 'Fwd Avg Bytes/Bulk',
    'fwd_pkts_b_avg': 'Fwd Avg Packets/Bulk',
    'fwd_blk_rate_avg': 'Fwd Avg Bulk Rate',
    'bwd_byts_b_avg': 'Bwd Avg Bytes/Bulk',
    'bwd_pkts_b_avg': 'Bwd Avg Packets/Bulk',
    'bwd_blk_rate_avg': 'Bwd Avg Bulk Rate',
    'subflow_fwd_pkts': 'Subflow Fwd Packets',
    'subflow_fwd_byts': 'Subflow Fwd Bytes',
    'subflow_bwd_pkts': 'Subflow Bwd Packets',
    'subflow_bwd_byts': 'Subflow Bwd Bytes',
    'init_fwd_win_byts': 'Init_Win_bytes_forward',
    'init_bwd_win_byts': 'Init_Win_bytes_backward',
    'fwd_act_data_pkts': 'act_data_pkt_fwd',
    'fwd_seg_size_min': 'min_seg_size_forward',
    'active_mean': 'Active Mean',
    'active_std': 'Active Std',
    'active_max': 'Active Max',
    'active_min': 'Active Min',
    'idle_mean': 'Idle Mean',
    'idle_std': 'Idle Std',
    'idle_max': 'Idle Max',
    'idle_min': 'Idle Min',
    'cwe_flag_count': 'CWE Flag Count'
}

data.rename(columns=rename_mapping, inplace=True)

# Save the DataFrame with predictions to a CSV file
data.to_csv('Data_with_predictions_LSTM.csv', index=False)